In [2]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams.update({"axes.grid":True,"grid.color":"black","grid.alpha":"0.25","grid.linestyle":"--"})
plt.rcParams.update({'font.size': 14})

from scipy import optimize

import sympy as sm
from IPython.display import display

1. AS-AD in open economy
2. Analytical solution
3. Numerical solution and calibration of model
4. Further analysis
- Adjustment to equilibrium
- Demand vs supply shocks
- Long run AS-AD
5. Conclusion